In [25]:
import pyarrow as pa
import pyarrow.parquet as pq
import librosa
from joblib import Parallel, delayed
import glob
from pathlib import Path
import re
import os

In [28]:
clean_dir = Path("/datasets/datasets_fullband/training_set/clean")

In [29]:
wav_fpath_list = librosa.util.find_files('/datasets/datasets_fullband/training_set/noisy')
len(wav_fpath_list)

60002

In [30]:
file_id_from_name = re.compile("fileid_(\d+)")
for wav_fpath in wav_fpath_list:
    filename = wav_fpath.split(os.sep)[-1]
    file_id = int(file_id_from_name.findall(filename)[0])
    clean_fpath = clean_dir / f"clean_fileid_{file_id}.wav"
    assert clean_fpath.exists()

In [23]:
wav_fpath_list_glob = glob.glob('/datasets/datasets_fullband/training_set/noisy/**.wav')
len(wav_fpath_list_glob)

60002

In [15]:
clean_dir = Path('/datasets/datasets_fullband/training_set') / "clean"

In [16]:
glob.glob(f"{clean_dir}/**.wav")

['/datasets/datasets_fullband/training_set/clean/clean_fileid_57032.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_16190.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_36877.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_57967.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_14374.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_40416.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_29380.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_36975.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_42841.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_47355.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_37009.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_34675.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_59549.wav',
 '/datasets/datasets_fullband/training

In [13]:
wav_fpath_list_glob[0:10]

['/datasets/datasets_fullband/training_set/clean/clean_fileid_0.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_1.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_10.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_100.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_1000.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_10000.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_10001.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_10002.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_10003.wav',
 '/datasets/datasets_fullband/training_set/clean/clean_fileid_10004.wav']

In [5]:
def librosa_load(fpath):
    return librosa.load(fpath, sr=16000)[0]

In [6]:
wav_waveform_list = Parallel(n_jobs=40)(delayed(librosa_load)(wav_fpath) for wav_fpath in wav_fpath_list)

In [7]:
# save path and waveform to pyarrow table
table = pa.Table.from_arrays([pa.array(wav_fpath_list), pa.array(wav_waveform_list)], names=['path', 'waveform'])

In [11]:
pa.write_table(table, 'training_set.parquet')

AttributeError: module 'pyarrow' has no attribute 'write_table'

In [7]:
# Write to file
with pa.OSFile('wav_fpath_list.arrow', 'wb') as sink:
    with pa.RecordBatchFileWriter(sink, batch.schema) as writer:
        writer.write_batch(batch)

In [8]:
# Random access
with pa.memory_map('wav_fpath_list.arrow', 'r') as source:
    with pa.RecordBatchFileReader(source) as reader:
        batch = reader.get_batch(0)
        print(batch)

pyarrow.RecordBatch
wav_fpath_list: string
